In [34]:
import pandas as pd
from collections import Counter
df = pd.read_csv('scrapeddata.csv')


# Figure out most common themes

In [64]:
rowcounter = 0
themelist = []
themeset = []
# turn strings that I scraped to actual lists 
for row in df.iterrows():
    #iterate over rows 
    selecteddata = df.iloc[rowcounter,:]
    
    themes = selecteddata['Themes'].strip('[]')

    themes = themes.replace('"', '').split()
    themelist = themelist + themes
    themeset = list(set(themeset + themelist))
    
    rowcounter += 1

In [68]:
for items in themelist:
    for characters in items:
        if characters == '-':
            characters = ' '
    


print(themelist)

['racism-and-slavery,', 'truth-and-justice,', 'family', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'family,', 'christianity', 'racism-and-slavery', 'racism-and-slavery,', 'truth-and-justice,', 'family', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'the-power-of-music', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'christianity,', 'the-power-of-music', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'truth-and-justice,', 'christianity', 'racism-and-slavery,', 'truth-and-justice,', 'christianity', 'racism-and-slavery,', 'truth-and-justice,', 'christianity', 'racism-and-slavery,', 'truth-and-justice', 'leadership,', 'idealism-vs-practicality,', 'colonial-society,', 'british-society', 'idealism-vs-practicality,', 'colonial-society', 'leadership,', 'british-society', 'militar

In [59]:
themeset

['race-racism-and-miscegenation',
 'plurality-and-the-power-of-the-individual-in-america,',
 'honor-loyalty-and-betrayal',
 'communication-versus-silence,',
 'commerce-and-exchange',
 'identity-and-illness,',
 'social-standing-and-pride',
 'youth-and-age',
 'alienation-and-control,',
 'endurance-vs-rebellion,',
 'identity-and-assumptions',
 'intertextuality',
 'truth-and-lies,',
 'memory-and-grief,',
 'talent-opportunity-work-and-luck',
 'the-power-of-thrift',
 'youth-maturity-and-growing-up,',
 'nature-v-nurture',
 'religion-and-politics,',
 'history-and-ethnic-identity',
 'vietnam-authorship-interpretation,',
 'labor-vs-productivity',
 'authenticity-and-femininity,',
 'the-power-of-symbols',
 'temptation-sin-and-redemption,',
 'conformity-and-success',
 'christianity-and-common-sense,',
 'the-true-and-the-essential',
 'social-networking-and-the-internet,',
 'love-obsession-and-pleasure',
 'storytelling-censorship-and-power,',
 'interracial-relationships',
 'slavery-and-race,',
 'iden